In [1]:
from collections import namedtuple
from itertools import chain, islice
import json

from lib.legiscan import (
    get_bill_meta, 
    get_bill_text, 
    extract_bill_contents, 
    BillDescriptor,
)

RESOLVER_MAP = '../artifacts/resolver_map.json'
TMP_PATH = '../tmp/legiscan'
BILLS_PATH = '../bills'

def retrieve_all(filename: str):
    
    mapper = {}
    with open(filename, 'r') as f:
        mapper = json.load(f)

    tuples = chain.from_iterable(
        (BillDescriptor(state, k, v) for k, v in m['bills'].items())
        for state, m 
        in mapper.items()
    )

    metas = ((t, get_bill_meta(t, TMP_PATH)) for t in tuples)
    responses = ((t, get_bill_text(t, TMP_PATH, meta)) for t, meta in metas)
    extracts = (extract_bill_contents(t, BILLS_PATH, resp) for t, resp in responses)
    return len(list(extracts))

download_count = retrieve_all(RESOLVER_MAP)
download_count

Missing response data ../bills/bill_text_response_SC_S0585.json


486